In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
from typing import List, Tuple
from openvino.runtime import Dimension, Core

sys.path.append("../generator")
from scratch_mark_generator import create_handwritten_circle, create_handwritten_cross, calculate_mask, create_handwritten_doodle, create_handwritten_tick, show_image


In [ ]:
def read_image(path: str) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)


def show_image(image: np.ndarray, color=cv2.COLOR_GRAY2RGB) -> None:
    if color is not None:
        rgb_image = cv2.cvtColor(image, color)
    else:
        rgb_image = image
    plt.figure(figsize=(10, 14))
    plt.axis('off')

    plt.imshow(rgb_image)


In [ ]:
image = read_image('exams/image--001.jpg')


In [ ]:
def detect_contours(image: np.ndarray, threshold: int = 80) -> np.ndarray:
    new_image = image.copy()
    lower_black = np.array([0])
    upper_black = np.array([threshold])
    mask = cv2.inRange(new_image, lowerb=lower_black, upperb=upper_black)
    black_cnt = cv2.findContours(
        mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2]
    return black_cnt


def find_squares(contours: np.ndarray, square_prop: float = 1.1) -> np.ndarray:
    def is_square(candidate: List[int]) -> bool:
        return max(candidate[2:])/min(candidate[2:]) <= square_prop
    return [contour for contour in contours if is_square(cv2.boundingRect(contour))]


def group_by_size(contours: np.ndarray, similarity_prop: float = 1.1) -> List[List[int]]:
    groupped = [[]]
    cont_idx, group_idx = 0, 0

    contours = drop_insignificant(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    while cont_idx < len(contours):
        if len(groupped[group_idx]) == 0 or cv2.contourArea(groupped[group_idx][0])/cv2.contourArea(contours[cont_idx]) < similarity_prop:
            groupped[group_idx].append(contours[cont_idx])
        else:
            groupped.append([contours[cont_idx]])
            group_idx += 1
        cont_idx += 1

    return groupped


def drop_insignificant(contours: np.ndarray, dropout: float = 400):
    return [contour for contour in contours if cv2.contourArea(contour) >= dropout]


def mark_contours(image: np.ndarray, contours: np.ndarray, frame_color: Tuple[int] = (0, 255, 0)) -> np.ndarray:
    if len(image.shape) == 2:
        new_image = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2RGB)
    else:
        new_image = image.copy()
    for m_area in contours:
        if type(m_area[0]) != int:
            (xg, yg, wg, hg) = cv2.boundingRect(m_area)
        else:
            (xg, yg, wg, hg) = m_area
        cv2.rectangle(new_image, (xg, yg), (xg+wg, yg+hg), frame_color, 2)
    return new_image


def calculate_rectangle(contours: np.ndarray, inside: bool = True) -> List[int]:
    contours = [cv2.boundingRect(contour) for contour in contours]
    right_x = [x[0] + x[2] for x in contours]
    bottom_y = [x[1] + x[3] for x in contours]
    if inside:
        first_fun, second_fun = max, min
    else:
        first_fun, second_fun = min, max
    x1 = first_fun(contours, key=lambda x: x[0])[0]
    y1 = first_fun(contours, key=lambda x: x[1])[1]

    x2 = second_fun(right_x)
    y2 = second_fun(bottom_y)
    w, h = abs(x1 - x2), abs(y1 - y2)
    return [[min(x1, x2), min(y1, y2), w, h]]


def calculate_personal_data(squares: List[int], answers: List[int]) -> List[int]:
    return [squares[0][:3] + [abs(answers[0][1] - squares[0][1])]]


In [ ]:
contours = detect_contours(image=image)

square_contours = find_squares(contours)
groupped_squares_contours = group_by_size(square_contours)[0]
rectangle_inside_squares = calculate_rectangle(groupped_squares_contours)

groupped_rectangle_contours = group_by_size(contours)[0]
rectangle_answers_connected = calculate_rectangle(
    groupped_rectangle_contours, inside=False)

bound_personal = calculate_personal_data(
    rectangle_inside_squares, rectangle_answers_connected)

image_with_marked_squares = mark_contours(image, rectangle_inside_squares)
image_with_marked_answers = mark_contours(
    image_with_marked_squares, rectangle_answers_connected, (255, 0, 0))
image_with_marked_personal = mark_contours(
    image_with_marked_answers, bound_personal, (0, 0, 255))

# show_image(image_with_marked_personal, None)


In [ ]:
x1, y1, w1, h1 = rectangle_answers_connected[0]
x2, y2, w2, h2 = bound_personal[0]
answers_image = image[y1:y1+h1, x1:x1+w1]
personal_image = image[y2:y2+h2, x2:x2+w2]


In [ ]:
# show_image(answers_image)
# show_image(personal_image)

def detect_lines(threshhold, kernel_size: Tuple[int]):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    detect = cv2.morphologyEx(threshhold, cv2.MORPH_OPEN, kernel, iterations=2)
    contours = cv2.findContours(detect, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours[0] if len(contours) == 2 else contours[1]

answers_image[answers_image <= 170] = 0
answers_image[answers_image > 170] = 255
new_ans = answers_image.copy()

thresh = cv2.threshold(new_ans, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

new_ans = cv2.cvtColor(new_ans, cv2.COLOR_GRAY2RGB)

horizontal_contours = detect_lines(thresh, (40, 1))
vertical_contours = detect_lines(thresh, (1, 30))

cv2.drawContours(new_ans, horizontal_contours, -1, (0,255,0), 2)
cv2.drawContours(new_ans, vertical_contours, -1, (255,0,0), 2);

show_image(new_ans, None)

In [ ]:
def calculate_box_coordinates(horizontal_contours, vertical_contours):
    def calculate_lines_boundaries(lines):
        boundaries = []
        for line in lines:
            x1 = min(line, key=lambda x: x[0])[0]
            y1 = min(line, key=lambda x: x[1])[1]
            x2 = max(line, key=lambda x: x[0])[0]
            y2 = max(line, key=lambda x: x[1])[1]
            boundaries.append(x1, y1, x2, y2)
        return boundaries
    
    def calculate_cross_points(first_line, second_line, perpendicular_lines: list) -> list:
        if first_line[2] < second_line[0]:
            return [] 
        horizontal_boundaries = calculate_lines_boundaries([first_line, second_line]) # horizontal
        perpendicular_boundaries = calculate_lines_boundaries(perpendicular_lines)
        cross_points = []
        for line_idx in range(len(perpendicular_boundaries)):
            vertical_boundaries = perpendicular_boundaries[line_idx] # vertical
            for hb in horizontal_boundaries:
                if hb[0] <= (vertical_boundaries[0] + vertical_boundaries[2])/2 <= hb[2] and \
                vertical_boundaries[1] <= (vertical_boundaries[1] + vertical_boundaries[3])/2 <= vertical_boundaries[3]:
                    cross_points.append([(vertical_boundaries[0] + vertical_boundaries[2])/2, (hb[1] + hb[3])/2])
        return cross_points
    
    def sort_horizontals(horizontal_contours):
        return sorted(horizontal_contours, key=lambda x: x[0])

    def sort_vertical(vertical_contours):
        temp_vertical = sorted(vertical_contours, key=lambda x: (x[0], x[1]))
        return temp_vertical[::2] + temp_vertical[1::2]

calculate_box_coordinates(horizontal_contours=horizontal_contours, vertical_contours=vertical_contours)

In [ ]:
def calculate_line_length(line: List[List[int]]):
    n_line = [l[0] for l in np.squeeze(line, axis=1)]
    return max(n_line) - min(n_line)

def split_horizontals_to_boxes(lines: List[List[List[int]]]) -> List[List[int]]:
    boxes = []
    idx = 0
    while idx < len(lines)//2:
        pair_of_lines = np.concatenate((lines[2*idx], lines[2*idx+1]), axis=0)
        x, y, w, h = cv2.boundingRect(pair_of_lines)
        boxes.append([x, y, w, h])
        idx += 1
    return boxes

In [ ]:
gray_convert_threshold = 150

pers_copy = personal_image.copy()
pers_copy[pers_copy <= gray_convert_threshold] = 0
pers_copy[pers_copy > gray_convert_threshold] = 255

thresh = cv2.threshold(pers_copy, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
pers_copy = cv2.cvtColor(pers_copy, cv2.COLOR_GRAY2RGB)
personal_horizontal = detect_lines(thresh, (30, 1))

personal_horizontal = sorted(personal_horizontal, key=calculate_line_length, reverse=True)
personal_horizontal_boxes = split_horizontals_to_boxes(personal_horizontal)
# pers_copy = mark_contours(personal_image, personal_horizontal_boxes)

# show_image(pers_copy, None)

In [ ]:

ie = Core()

letters = '~ !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz{|}~£'

recognition_model = ie.read_model(model="models/handwritten-english-recognition-0001.xml", weights="models/handwritten-english-recognition-0001.bin")

compiled_recognition_model = ie.compile_model(model=recognition_model, device_name='AUTO')
recognition_input_layer = compiled_recognition_model.input(0)
recognition_output_layer = compiled_recognition_model.output(0)

# show_image(personal_image)

In [ ]:
def remove_lines(image: np.ndarray, lines: List[List[int]], color = 255) -> np.ndarray:
    for line in lines:
        line = np.squeeze(line, axis=1)
        x1 = min(line, key=lambda x: x[0])[0]
        y1 = min(line, key=lambda x: x[1])[1]
        x2 = max(line, key=lambda x: x[0])[0]
        y2 = max(line, key=lambda x: x[1])[1]
        cv2.rectangle(image, (x1, y1), (x2, y2), color, -1)

def decode(preds, character_list):
    """ convert text-index into text-label. """
    texts = []
    # Select max probability (greedy decoding) then decode index to character
    preds_index = np.argmax(preds, 2) # WBD - > WB
    preds_index = preds_index.transpose(1, 0) # WB -> BW
    preds_index_reshape = preds_index.reshape(-1) # B*W

    char_list = []
    odds_list = []
    for i in range(len(preds_index_reshape)):
        if preds_index_reshape[i] != 0 and (not (i > 0 and preds_index_reshape[i - 1] == preds_index_reshape[i])):
            char_list.append(character_list[preds_index_reshape[i]])
            odds_list.append(max(preds[preds_index_reshape[i]][0]))
    text = ''.join(char_list)
    texts.append(text)
    return texts # , odds_list

In [ ]:
gray_convert_threshold = 175

gray_personal_copy = personal_image.copy()
gray_personal_copy[gray_personal_copy <= gray_convert_threshold] = 0
gray_personal_copy[gray_personal_copy > gray_convert_threshold] = 255
show_image(gray_personal_copy)

personal_horizontal = detect_lines(thresh, (20, 1))

remove_lines(gray_personal_copy, personal_horizontal)

k1 = (4,1)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, k1)
i_b, i_c, i_h, i_w = recognition_input_layer.partial_shape
i_h, i_w = i_h.get_length(), i_w.get_length()
for box in personal_horizontal_boxes:
    x, y, w, h = box
    crop = gray_personal_copy[y:y+h, x:x+w]
    thresh = cv2.threshold(crop, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    personal_vertical = detect_lines(thresh, (1, 14))
    personal_horizontal = detect_lines(thresh, (30, 1))
    remove_lines(crop, personal_vertical)
    remove_lines(crop, personal_horizontal)
    ratio = i_h/h
    crop = cv2.resize(crop, (int(ratio*w), i_h), interpolation=cv2.INTER_AREA)[None]
    crop = np.pad(crop, ((0, 0), (0, 0), (0, i_w - int(ratio*w))), mode='edge')[None]
    results = compiled_recognition_model([crop])[recognition_output_layer]
    results = decode(results, letters)
    results = np.squeeze(results)
    
    show_image(crop[0][0])
    print(results)

In [ ]:
# def preprocess_input(image_name, height, width):
#     src = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
#     ratio = float(src.shape[1]) / float(src.shape[0])
#     tw = int(height * ratio)
#     rsz = cv2.resize(src, (tw, height), interpolation=cv2.INTER_AREA)
#     # [h,w] -> [c,h,w]
#     img = rsz[None, :, :]
#     _, h, w = img.shape
#     # right edge padding
#     pad_img = np.pad(img, ((0, 0), (0, height - h), (0, width - w)), mode='edge')
#     return pad_img

# input = preprocess_input('exams/handwritten_english_test.png', i_h, i_w)[None]
# results = compiled_recognition_model([input])[recognition_output_layer]
# print(decode(results, letters))